In [ ]:
import os
import subprocess
import requests
from tqdm.notebook import tqdm

def download_file(url, filename, max_retries=3):
    for attempt in range(max_retries):
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            total_size = int(response.headers.get('content-length', 0))
            block_size = 1024  # 1 KB
            
            with open(filename, 'wb') as file, tqdm(
                desc=f"Attempt {attempt + 1}: {filename}",
                total=total_size,
                unit='iB',
                unit_scale=True,
                unit_divisor=1024,
            ) as progress_bar:
                for data in response.iter_content(block_size):
                    size = file.write(data)
                    progress_bar.update(size)
            return True
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {str(e)}")
            if attempt == max_retries - 1:
                raise e
    return False

def run_command(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, text=True)
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())
    rc = process.poll()
    return rc

print("Starting ComfyUI setup...")

# Update or clone ComfyUI
comfyui_path = '/workspace/ComfyUI'

# Create necessary directories
directories = [
    '/workspace/ComfyUI/custom_nodes',
    '/workspace/ComfyUI/models/checkpoints',
    '/workspace/ComfyUI/models/controlnet/sdxl',
    '/workspace/ComfyUI/models/unet',
    '/workspace/ComfyUI/models/upscale_models',
    '/workspace/ComfyUI/models/upscale_models/ClearRealityV1',
    '/workspace/ComfyUI/models/clip',
    '/workspace/ComfyUI/models/vae',
    '/workspace/ComfyUI/models/loras'
]
for directory in directories:
    os.makedirs(directory, exist_ok=True)

# Clone repositories to custom_nodes
custom_nodes_repos = [
    "https://github.com/ltdrdata/ComfyUI-Impact-Pack.git",
    "https://github.com/city96/ComfyUI-GGUF.git",
    "https://github.com/pythongosssss/ComfyUI-Custom-Scripts.git",
    "https://github.com/Suzie1/ComfyUI_Comfyroll_CustomNodes.git",
    "https://github.com/rgthree/rgthree-comfy.git",
    "https://github.com/giriss/comfy-image-saver.git",
    "https://github.com/Acly/comfyui-tooling-nodes.git",
    "https://github.com/spacepxl/ComfyUI-Florence-2.git",
    "https://github.com/kijai/ComfyUI-KJNodes.git",
    "https://github.com/kijai/ComfyUI-Florence2.git",
    "https://github.com/kijai/ComfyUI-segment-anything-2.git",
    "https://github.com/yolain/ComfyUI-Easy-Use.git",
    "https://github.com/KoreTeknology/ComfyUI-Universal-Styler.git",
    "https://github.com/un-seen/comfyui-tensorops.git",
    "https://github.com/Xclbr7/ComfyUI-Merlin.git",
    "https://github.com/digitaljohn/comfyui-propost.git",
    "https://github.com/BlenderNeko/ComfyUI_Noise.git",
    "https://github.com/WASasquatch/was-node-suite-comfyui.git",
    "https://github.com/Clybius/ComfyUI-Latent-Modifiers.git",
    "https://github.com/chrisgoringe/cg-use-everywhere.git",
    "https://github.com/ssitu/ComfyUI_UltimateSDUpscale.git",
    "https://github.com/city96/ComfyUI-GGUF.git",
]

print("\nCloning custom node repositories...")
for repo in custom_nodes_repos:
    repo_name = repo.split('/')[-1].replace('.git', '')
    command = f'git clone {repo} /workspace/ComfyUI/custom_nodes/{repo_name}'
    run_command(command)

# Download files only if they don't exist
# Download files only if they don't exist
files_to_download = {
    '/workspace/ComfyUI/models/checkpoints/wildcardx-xl-turbo.safetensors': 'https://civitai.com/api/download/models/329685?type=Model&format=SafeTensor&size=full&fp=fp16',
    '/workspace/ComfyUI/models/controlnet/sdxl/OpenPoseXL2.safetensors': 'https://huggingface.co/thibaud/controlnet-openpose-sdxl-1.0/resolve/main/OpenPoseXL2.safetensors',
    '/workspace/ComfyUI/models/controlnet/sdxl/mistoLine_rank256.safetensors': 'https://huggingface.co/TheMistoAI/MistoLine/resolve/main/mistoLine_rank256.safetensors',
    '/workspace/ComfyUI/models/upscale_models/ClearRealityV1/4x-ClearRealityV1.pth': 'https://github.com/agsz5/comfyUI_setup/raw/main/4x-ClearRealityV1.pth',
    '/workspace/ComfyUI/models/loras/LineAniRedmondV2-Lineart-LineAniAF.safetensors': 'https://huggingface.co/artificialguybr/LineAniRedmond-LinearMangaSDXL-V2/resolve/fa60a2a60e93448e609e5bf1cc8933bd30d4b7e2/LineAniRedmondV2-Lineart-LineAniAF.safetensors'
}

print("\nChecking and downloading model files...")
for filename, url in files_to_download.items():
    if not os.path.exists(filename):
        print(f"Downloading {filename}...")
        download_file(url, filename)
    else:
        print(f"{filename} already exists. Skipping download.")

print("\nSetup completed successfully!")